In [1]:
__author__ = 'Derek Qi'
# Doing portfolio backtest and generates output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from time import time
import pickle
import os
import sklearn.preprocessing as pp

from setup.univ_setup import *
from factor_mining.combine_factors import *
from factor_mining.factor_model_regression import *
from factor_mining.factor_preprocessing import *

from factor_model.stock_ret_est import GenReturn 
from GenPosition import *
from backtest_main import *
from performance_analysis.pa_core import *

from factor_mining.factors.momentum import *

from factor_mining.Mark0 import * # This is alpha file

In [2]:
### universe setup ###
print('Setup R3000 universe')
datadir = '/home/derek-qi/Documents/R3000_Data/data/r3000/'
start = time()
if os.path.exists(datadir + 'univ.pkl'):
    print('use existing binary file')
    with open(datadir + 'univ.pkl', 'rb') as univ_fh:
        univ = pickle.load(univ_fh)
    
else:
    print('construct from csv')
    big_table_dir = datadir + 'big_table_full_v3.csv'
    univ = univ_setup(big_table_dir)
    filt_by_name(univ) # This is fucking slow！
    with open(datadir + 'univ.pkl','wb') as fh:
        pickle.dump(univ, fh)
end = time()
print('%f seconds' % (end - start))

Setup R3000 universe
use existing binary file
0.513083 seconds


In [4]:
### model configuration ###
print('Calculate factors')
# define and calculate all factors
factors = alpha_four_factors(univ)
# concat into factor exposure matrices
factor_exp_mat = combine_factors(factors)
# Preprocessing factor expsure matrices
print('Scale each factor')
factor_exp_mat = process_batch(factor_exp_mat, pp.scale)
print('Winsorize with +/- 3 std')
factor_exp_mat = process_batch(factor_exp_mat, winsorize_std)
print('Done')

Calculate factors
Scale each factor


/home/derek-qi/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Winsorize with +/- 3 std
Done


In [5]:
# const setup
factor_names = [k for k in factors.keys()]
N_f = len(factor_names)
datelst = sorted(factor_exp_mat.keys())
N_T = len(datelst)

In [ ]:
# Run backtest
# Calc stock returns
rebal = 1
ret_series = log_return(univ, -rebal)

In [ ]:
dstart = datetime(2015, 1, 1)
dend = datetime(2015, 12, 31)
print('Start backtest from %s to %s' % (dstart.strftime('%Y/%m/%d'), dend.strftime('%Y/%m/%d')))
ts = time()
# ptfl_fin, pnl = backtest_batch(univ, factor_exp_mat, ret_series, dstart, dend, silent=False)
ptfl_fin, pnl = backtest_multi_period_rebalance(univ, factor_exp_mat, ret_series, dstart, dend, rebal, silent=False)
te = time()
print(te - ts)

Start backtest from 2015/01/01 to 2015/12/31
2015-01-02 00:00:00
Pool size: 559
        ticker    weight   log_ret
0    X1288453D  0.200004  0.078904
12         ABC  0.067904  0.027691
31         ALL  0.011251  0.004269
163        DRI  0.007188  0.034346
275       JACK  0.025608  0.060300
293         KR  0.005740  0.035716
307        LUV  0.001276 -0.045529
321        MHK  0.006153  0.024161
383         PL  0.025003  0.000861
395        RAX  0.115307 -0.005512
405        RHI  0.053404  0.011679
421       SBAC  0.015894 -0.001082
422        SBH  0.036892 -0.024906
424       SCSS  0.007552  0.039764
435        SHW  0.015310  0.044639
464        TGT  0.005506  0.014497
493       ULTA  0.200004  0.036613
546       ZLTQ  0.200004  0.086233
Period log pnl: 0.044503
2015-01-09 00:00:00
Pool size: 790
        ticker    weight   log_ret
0    X1288453D  0.024078  0.015683
24         AEC  0.004333  0.026995
52        ANAC  0.180838 -0.001798
55        ANGO  0.003409 -0.004758
80        AVID  0.18

In [ ]:
pnl.columns = ['Date', 'Pnl']
# simple_pa(pnl)
pmfc = (cagr(pnl), vol(pnl), sharpe_ratio(pnl), max_drawdown(pnl), drawdown_length(pnl))
print('CAGR:%f \nVolatility:%f\nPeriodical_Sharpe_ratio:%f\nMax drawdown: %f\nDrawdown length: %f\n' % pmfc)

plot_nav(pnl)